## Bracket Challenge Predictor

Reading data

In [5]:
import pandas as pd
import numpy as np
import matplotlib as plt

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

from xgboost import XGBRegressor

In [110]:
team_matchups = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Tournament Matchups.csv")

away_stats = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Barttorvik Away.csv")
home_stats = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Barttorvik Home.csv")
neutral_stats = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Barttorvik Neutral.csv")
conf_perf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Conference Stats.csv")
team_res = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Team Results.csv")
team_v_ranked = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/TeamRankings.csv")
team_shot_dis = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Shooting Splits.csv")

ken_def = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/INT _ KenPom _ Defense.csv")
ken_off = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/INT _ KenPom _ Offense.csv")
ken_eff = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/INT _ KenPom _ Efficiency.csv")
ken_height = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/INT _ KenPom _ Height.csv")
ken_mis = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/INT _ KenPom _ Miscellaneous Team Stats.csv")
ken_pt_dis = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/INT _ KenPom _ Point Distribution.csv")
ken_summ = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/INT _ KenPom _ Summary.csv")
ken_conf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/DEV _ March Madness.csv")

conf_mapping = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/REF _ NCAAM Conference and ESPN Team Name Mapping.csv")

cur_coaches = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/REF _ Current NCAAM Coaches.csv")
coach_perf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Coach Results.csv")

seed_res = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Seed Results.csv")
upset_count = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Upset Count.csv")
upset_inf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Upset Seed Info.csv")

/var/folders/g4/t1q9yqqx7h36p5_2jzq23jsw0000gn/T/ipykernel_28025/2544293188.py:18: DtypeWarning: Columns (144) have mixed types. Specify dtype option on import or set low_memory=False.
  ken_conf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/DEV _ March Madness.csv")


since ken_ datasets don't have TEAMNO var will first create seperate lookup table to create a column for its respective id

In [100]:
team_ids = team_matchups[['YEAR','TEAM','TEAM NO']].drop_duplicates()

ken_def.rename(columns={'Season':'YEAR','TeamName':'TEAM'},inplace=True)
ken_off.rename(columns={'Season':'YEAR','TeamName':'TEAM'},inplace=True)
ken_eff.rename(columns={'Season':'YEAR','Team':'TEAM'},inplace=True)
ken_height.rename(columns={'Season':'YEAR','TeamName':'TEAM'},inplace=True)
ken_mis.rename(columns={'Season':'YEAR','TeamName':'TEAM'},inplace=True)
ken_pt_dis.rename(columns={'Season':'YEAR','TeamName':'TEAM'},inplace=True)
ken_summ.rename(columns={'Season':'YEAR','TeamName':'TEAM'},inplace=True)


ken_def = ken_def.merge(team_ids,on=['YEAR','TEAM'],how='left')
ken_off = ken_off.merge(team_ids,on=['YEAR','TEAM'],how='left')
ken_eff = ken_eff.merge(team_ids,on=['YEAR','TEAM'],how='left')
ken_height = ken_height.merge(team_ids,on=['YEAR','TEAM'],how='left')
ken_mist = ken_mis.merge(team_ids,on=['YEAR','TEAM'],how='left')
ken_pt_dis = ken_pt_dis.merge(team_ids,on=['YEAR','TEAM'],how='left')


In [101]:
conf_ids = conf_perf[['YEAR','CONF','CONF ID']].drop_duplicates()

ken_conf.rename(columns={'Season':'YEAR','Short Conference Name':'CONF'},inplace=True)

ken_conf = ken_conf.merge(conf_ids,on=['YEAR','CONF'],how='left')

In [112]:
team_matchups.head()

,YEAR,BY YEAR NO,BY ROUND NO,TEAM NO,TEAM,SEED,ROUND,CURRENT ROUND,SCORE
0,2024,2014,2014,1067,Connecticut,1,1,64,91
1,2024,2013,2013,1026,Stetson,16,64,64,52
2,2024,2012,2012,1060,Florida Atlantic,8,64,64,65
3,2024,2011,2011,1036,Northwestern,9,32,64,77
4,2024,2010,2010,1029,San Diego St.,5,16,64,69


In [ ]:
temp_away = away_stats.loc[:,['YEAR','TEAM NO','TEAM','BADJ EM','BADJ O',
                'BADJ D','WIN%','EFG%','EFG%D','FTR','FTRD',
                'BADJ T','TOV%','TOV%D','RAW T','PPPO','PPPD','ELITE SOS','WAB']]
temp_home = home_stats.loc[:,['YEAR','TEAM NO','TEAM','BADJ EM','BADJ O',
                'BADJ D','WIN%','EFG%','EFG%D','FTR','FTRD',
                'BADJ T','TOV%','TOV%D','RAW T','PPPO','PPPD','ELITE SOS','WAB']]
temp_neutral = neutral_stats.loc[:,['YEAR','TEAM NO','TEAM','BADJ EM','BADJ O',
                'BADJ D','WIN%','EFG%','EFG%D','FTR','FTRD',
                'BADJ T','TOV%','TOV%D','RAW T','PPPO','PPPD','ELITE SOS','WAB']]

#adds _A to all columns to represent Away
temp_away.columns = list(temp_away.columns[:3]) + [col + "_A" for col in temp_away.columns[3:]]

#_H for Home
temp_home.columns = list(temp_home.columns[:3]) + [col + "_H" for col in temp_home.columns[3:]]

#_N for Neutral
temp_neutral.columns = list(temp_neutral.columns[:3]) + [col + "_N" for col in temp_neutral.columns[3:]]


,YEAR,TEAM NO,TEAM,BADJ EM_A,BADJ O_A,BADJ D_A,WIN%_A,EFG%_A,EFG%D_A,FTR_A,FTRD_A,BADJ T_A,TOV%_A,TOV%D_A,RAW T_A,PPPO_A,PPPD_A,ELITE SOS_A,WAB_A
0,2024,1079,Akron,4.3,109.1,104.8,53.846154,52.8,50.5,29.1,34.3,66.7,16.9,16.3,66.7,1.095,1.046,15.936,-3.10
1,2024,1078,Alabama,16.0,124.5,108.5,50.000000,54.3,55.2,35.7,48.9,73.8,16.5,14.4,75.2,1.161,1.199,54.469,1.40
2,2024,1077,Arizona,24.1,123.6,99.5,63.636364,53.1,52.3,37.6,34.0,72.5,15.9,16.4,72.2,1.147,1.070,42.102,1.80
3,2024,1076,Auburn,21.1,118.9,97.8,50.000000,52.8,46.5,31.9,42.8,69.0,15.8,15.0,70.1,1.139,1.039,44.169,0.46
4,2024,1075,Baylor,18.4,117.4,99.0,44.444444,49.9,53.6,36.1,32.6,64.0,16.6,17.8,64.5,1.082,1.068,51.232,0.60


Created a column on who won each matchup for every game in the dataset. Will now add columns of interest to this dataset and will try to do feature engineering as to model who'd win each matchup.

In [139]:
wins = []
for i in range(len(team_matchups)//2):
    if team_matchups.iloc[i,8] > team_matchups.iloc[i+1,8]:
        wins.append(1)
        wins.append(0)
    else:
        wins.append(0)
        wins.append(1)
    i += 2
team_matchups['WIN'] = wins
        
team_matchups.head()

,YEAR,BY YEAR NO,BY ROUND NO,TEAM NO,TEAM,SEED,ROUND,CURRENT ROUND,SCORE,WIN
0,2024,2014,2014,1067,Connecticut,1,1,64,91,1
1,2024,2013,2013,1026,Stetson,16,64,64,52,0
2,2024,2012,2012,1060,Florida Atlantic,8,64,64,65,0
3,2024,2011,2011,1036,Northwestern,9,32,64,77,1
4,2024,2010,2010,1029,San Diego St.,5,16,64,69,0
